In [1]:
import numpy as np
import matplotlib.pyplot as plt
from multiprocessing import Pool
from tqdm import tqdm
from Behaviour_pi import Behavioural
from Behaviour_pi import Policy
import gym
import time
import flexible_bus
from policy import model_1, model_2  
import json
import matplotlib.pyplot as plt
from concurrent.futures import ProcessPoolExecutor
from functools import partial
import os
import torch
import torch.nn as nn
import pandas as pd


In [ ]:
max_epochs = 1000000
step = 5000
min_epochs = 1000
alpha = 0.05  # Adjust based on desired sensitivity
traj_num = 1000000
traj_len = 5
gamma = 0.99
converge = 0.0001
confidence_level = 0.9
# base_policy = [0.5,0.5]
eval_pi = torch.load("./Policys/Perturbed_model_0.pth")
converge_epoch_ls = []
# for i in tqdm(range(9),desc="Outer_loop"):
#     for j in tqdm(range(9),desc="Inter_loop"): 
base_pi = Behavioural()
base_pi.load_state_dict(torch.load("./Behavioural_model.pth"))
traj_list = pd.read_parquet("./Traject/trajectories.pq")
return_list = np.loadtxt("./return_list.txt")


In [ ]:
return_list = traj_list.iloc[:5, 1].to_numpy()
up_b, low_b = max(return_list), min(return_list)
up_b,low_b

(np.float64(37.776383439), np.float64(1.93089501))

In [17]:
iw_ls = []
wr_ls=[] #weighte return list
# up_b, low_b = max(return_list), min(return_list)
batch_size = 5
for j in range(batch_size):
    for i in range(traj_len):
        obs = traj_list.iloc[j][f"obs_{i}"]
        a = traj_list.iloc[j][f"direct_action_{i}"]
        obs = torch.tensor(obs,dtype=torch.float32)
        eval_out = eval_pi(obs)
        base_out = base_pi(obs)
        eval_policy = [[1-eval_out[0].item(),eval_out[0].item()],[1-eval_out[1].item(),eval_out[1].item()]]
        base_policy = [[1-base_out[0].item(),base_out[0].item()],[1-base_out[1].item(),base_out[1].item()]]
        importance_weights =((eval_policy[0][a[0]] * eval_policy[1][a[1]]) /(base_policy[0][a[0]] * base_policy[1][a[1]]))
        iw_ls.append(importance_weights)
    importance_weight = np.prod(iw_ls)
    norm_return = (traj_list.iloc[j]["return"] - low_b) / (up_b - low_b)
    wr = norm_return*importance_weight
    wr_ls.append(norm_return)
wr_ls       

[np.float64(0.42990124881470054),
 np.float64(0.32128364055663894),
 np.float64(0.3273018313933267),
 np.float64(0.40874862227142345),
 np.float64(0.3483245076917013)]

In [ ]:
up_b, low_b = max(return_list), min(return_list)

"""
Func to use the collected trajectories to estimate the evaluation policy

Args:
    traj (_type_): dict
    eval_policy (_type_): _description_
    base_policy (_type_): _description_
    up_b (_type_): _description_
    low_b (_type_): _description_
"""
traj = traj_list.iloc[0]
obs = torch.tensor(obs,dtype=torch.float32)
[p1, p2] = base_policy(obs)
p1 = p1.item()
p2 = p2.item()
direct_action = [p1, p2]


eval_policy = [[1-eval_policy[0],eval_policy[0]],[1-eval_policy[1],eval_policy[1]]]
base_policy = [[1-base_policy[0],base_policy[0]],[1-base_policy[1],base_policy[1]]]
actions = traj["actions"]  # Assume it's a list of lists
importance_weights = [
    (eval_policy[0][a[0]] * eval_policy[1][a[1]]) /
    (base_policy[0][a[0]] * base_policy[1][a[1]])
    for a in actions
]
importance_weight = np.prod(importance_weights)
norm_return = (traj["return"] - low_b) / (up_b - low_b)